# CUSTOM LAYERS

## Used to define our own layers
### -> like the Dense layer can be defined by using logic Y = mX + c

In [1]:
# Import Dependencies
import tensorflow as tf
from tensorflow.keras.layers import Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

## Defining a Custom Dense Layer

In [2]:
# Defining The Custom Layer For Dense Layer
class NeuralDense(Layer):
    def __init__(self, outputUnits, activation):
        super(NeuralDense, self).__init__()
        self.outputUnits = outputUnits
        self.activation = activation
    
    # Layer Weights and Bias to learn
    def build(self, inputFeaturesShape):
        # Initialize weights and biases
        self.w = self.add_weight(
            shape=(inputFeaturesShape[-1], self.outputUnits),
            initializer='random_normal',
            trainable=True,
            name='weights'
        )
        self.b = self.add_weight(
            shape=(self.outputUnits,),
            initializer='random_normal',
            trainable=True,
            name='biases'
        )

    # Logic -> Y = mX + c => wX + b
    # wX => (B, F) * (F, Out) -> Matrix Multiplication
    # b => (B, Out)
    # => Y = ((B, F) * (F, Out)) + (B, Out)
    def call(self, inputFeatures):
        if self.activation=='relu':
            return tf.nn.relu(tf.linalg.matmul(inputFeatures, self.w) + self.b)
        elif self.activation=='sigmoid':
            return tf.math.sigmoid(tf.linalg.matmul(inputFeatures, self.w) + self.b)
        else:
            return tf.linalg.matmul(inputFeatures, self.weight) + self.b
            


## Using The Custom Dense Layer With Sequential API

In [4]:
# Using Sequential API
IMAGE_SIZE=224
model = tf.keras.Sequential([
    Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),

    Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Flatten(),

    # Using the Custom Dense Layer
    NeuralDense(100, activation='relu'),
    BatchNormalization(),
    NeuralDense(10, activation='relu'),
    BatchNormalization(),
    NeuralDense(1, activation='sigmoid'),
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ neural_dense_3 (NeuralDense)    │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ neural_dense_4 (NeuralDense)    │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ neural_dense_5 (NeuralDense)    │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)